In [ ]:
!pip install torch -Uq

In [ ]:
!pip install transformers -Uq

In [ ]:
!pip install tiktoken -Uq

In [ ]:
!pip install datasets -Uq

In [ ]:
!pip install matplotlib -Uq

In [ ]:
import json

class Tokenizer:
  def __init__(self, vocab_path):
    with open(vocab_path, "r", encoding="utf-8") as f:
      self.vocab = json.load(f)
      self.reverse_vocab = {v: k for k, v in self.vocab.items()}

  def encode(self, text):
    tokens = []

    for word in text.split():
      i = 0
      while i < len(word):
        found_match = False
        for j in range(len(word), i, -1):
          subword = word[i:j]
          if subword in self.vocab:
            tokens.append(self.vocab[subword])
            i = j
            found_match = True
            break
        if not found_match:
          tokens.append(self.vocab.get("<unk>", 0))
          i += 1
      tokens.append(self.vocab.get(" ", 1))
    if tokens:
      tokens.pop()
    return tokens

  def tokenize(self, text):
    token_ids = self.encode(text)
    return [self.reverse_vocab[id] for id in token_ids]

  def decode(self, ids):
    text = ""
    for id in ids:
      text += self.reverse_vocab.get(id, "<unk>")
    return text

In [ ]:
tokenizer = Tokenizer("tokenizer_1.json")

In [ ]:
import plotly.graph_objects as go
import plotly.offline

In [ ]:
def plot_dots(sentences_data, title, dims=[0, 1, 2]):
  data = [
    go.Scatter3d(
      x=sentence_data["words"][:, dims[0]],
      y=sentence_data["words"][:, dims[1]],
      z=sentence_data["words"][:, dims[2]],
      mode="markers+text",
      marker=dict(
        size=6,
        color=sentence_data["color"],
      ),
      text=sentence_data["labels"],
      hoverinfo="text",
    ) for sentence_data in sentences_data
  ]

  layout = go.Layout(
    scene=dict(
      xaxis_title="Sertlik",
      yaxis_title="Parlaklık",
      zaxis_title="Kırmızılık",
    ),
    title=title,
  )

  fig = go.Figure(data=data, layout=layout)
  plotly.offline.iplot(fig)

In [ ]:
data = "the capital of the united states"
target = "capital of the united stetes is"

In [ ]:
ids = tokenizer.encode(data)

len(ids), ids

(12, [0, 61, 1, 61, 2, 61, 0, 61, 3, 61, 4, 58])

In [ ]:
ids_t = tokenizer.encode(target)

len(ids_t), ids_t

(16, [1, 61, 2, 61, 0, 61, 3, 61, 58, 62, 62, 62, 62, 58, 61, 5])

Context length, bir dil modelinin aynı anda işleyebildiği en uzun giriş + çıkış token toplamıdır.

In [ ]:
context_length = 32 #Gpt-4o 128k

Eğer Context length dolmazsa sonu "<<c>ped>" ile doldurulur